In [1]:
import numpy as np
import pandas as pd
from tqdm import tqdm_notebook as tqdm

In [2]:
!python processing_dataset.py

X shape is (69623, 46) 
Y shape is (69623, 1)


In [26]:
Xtr = pd.read_csv('./x_train.csv', index_col=0).values
ytr = pd.read_csv('./y_train.csv', index_col=0).values.ravel()

Xts = pd.read_csv('./x_test.csv', index_col=0).values
yts = pd.read_csv('./y_test.csv', index_col=0).values.ravel()

In [4]:
n_features = Xtr.shape[1]

### gaussian RBF

In [5]:
def fi(mu, sigma, vectorized=False):
    def f(x):
        return np.exp(
            -0.5 * (x-mu).T.dot(
                np.dot(np.linalg.inv(sigma),(x-mu))
            )
        )
    def f_vec(x):
        return np.exp(
            -0.5* np.sum(
                (x-mu).dot(
                np.linalg.inv(sigma)) * (x-mu),
                axis=1)
            )
    return f_vec if vectorized else f

In [6]:
M = [(np.arange(n_features) == i).astype('uint8')+np.random.uniform(0,0.5,size=n_features) for i in range(n_features)]

sigmas = [np.diag(m) for m in M] + [np.eye(n_features)]

In [7]:
mu = np.zeros(n_features)

### constructing Fi matrix

In [8]:
fi_j = [fi(mu, S, vectorized=True) for S in sigmas]

In [61]:
Fi = np.matrix([f(Xtr) for f in fi_j]).T
Fi = np.c_[np.ones((Fi.shape[0],1)),Fi] # adding bias term\
n_samples, n_features = Fi.shape

### adding **$\lambda$**-regularizer

In [63]:
lambda_ = 6

### finding closed-form solution

In [64]:
if len(ytr.shape) != 2:
    ytr = np.matrix(ytr).T

In [65]:
inv = np.linalg.inv

In [66]:
W = inv(
    	np.eye(n_features) * lambda_ + Fi.T @ Fi
	) @ Fi.T @ ytr

In [67]:
from sklearn.metrics import mean_squared_error as MSE

In [68]:
MSE(ytr, Fi @ W)

0.31323936169366573

---
### Task 2 Gradient descent approach

In [217]:
class SGDLinRegression():
    def __init__(self, learning_rate=3e-4, regularizer=1):
        self.lr = learning_rate
        self.lambda_ = regularizer
        
    def step(self, W, X_train, Y_train):
        grad = (X_train @ W - Y_train).T @ X_train
        update = 1 / len(Y_train) * (grad + self.lambda_ * W.T)
        return W - self.lr * update.T
    
    def test(self, W, X, Y):
        return MSE(Y, X @ W)
    
    def predict(self, X):
        return X @ W
        
    def fit(self, X_train, Y_train, valid_set=None, n_epochs=1e4, verbose=False):
        W = np.random.uniform(-1, 1 ,size=n_features).reshape(-1,1)
        epochs = tqdm(range(n_epochs))
        for i in epochs:
            W = self.step(W, X_train, Y_train)
            if valid_set:
                epochs.set_description(
                    f"""epoch {i}: 
                    train_error={self.test(W,X_train,Y_train):.4f}
                    valid_error={self.test(W,valid_set[0],valid_set[1]):.4f}"""
                )
            else:
                epochs.set_description(
                    f"epoch {i}: MSE={self.test(W,X_train,Y_train):.4f}"
                )

In [218]:
model = SGDLinRegression(learning_rate=1e-2)

In [219]:
model.fit(Fi,ytr, verbose=True, n_epochs=1000)

HBox(children=(IntProgress(value=0, max=1000), HTML(value='')))